In [1]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
%reset -f

In [3]:
# Step 0: Install All Necessary Libraries
# -------------------------------------------------------------------
print("⚙️ Installing required libraries...")
print("✅ Libraries are ready.")
print("-" * 50)


# Step 1: Load All Data from the Official Server Path
# -------------------------------------------------------------------
import pandas as pd
import numpy as np
import xgboost as xgb
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from huggingface_hub import hf_hub_download

print("📂 Loading all source data files from the server...")
REPO_ID = "lainmn/AgentDS-Healthcare"
admissions_train = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/admissions_train.csv", repo_type="dataset"))
patients = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/patients.csv", repo_type="dataset"))
ed_cost_train = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/ed_cost_train.csv", repo_type="dataset"))
admissions_test = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/admissions_test.csv", repo_type="dataset"))
ed_cost_test = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/ed_cost_test.csv", repo_type="dataset"))
with open(hf_hub_download(REPO_ID, "Healthcare/discharge_notes.json", repo_type="dataset"), 'r') as f:
    discharge_notes = json.load(f)
notes_df = pd.DataFrame(discharge_notes)
print("✅ All data files loaded successfully.")
print("-" * 50)


# Step 2: Define Feature Engineering and Prepare Full Datasets
# -------------------------------------------------------------------
def feature_engineer(df):
    """Applies our tabular feature engineering steps."""
    categorical_cols = ['sex', 'insurance', 'primary_dx']
    df = pd.get_dummies(df, columns=categorical_cols, dummy_na=False, dtype=int)
    df['is_weekend_discharge'] = df['discharge_weekday'].isin([6, 7]).astype(int)
    df['avg_cost_per_ed_visit'] = df['prior_ed_cost_5y_usd'] / df['prior_ed_visits_5y']
    df['avg_cost_per_ed_visit'] = df['avg_cost_per_ed_visit'].replace([np.inf, -np.inf], 0).fillna(0)
    return df

print("🧠 Preparing the full training dataset (Tabular + NLP)...")
# Prepare Training Data
train_df = pd.merge(admissions_train, patients, on='patient_id', how='left')
train_df = pd.merge(train_df, ed_cost_train, on='patient_id', how='left')
train_df_with_notes = pd.merge(train_df, notes_df, on='admission_id', how='left')
train_df_with_notes['note'] = train_df_with_notes['note'].fillna('')
train_df_featured = feature_engineer(train_df_with_notes.copy())

# NLP Feature Engineering (TF-IDF)
tfidf = TfidfVectorizer(max_features=200, stop_words='english')

# Corrected method from .fit_ to .fit_transform
X_notes_train = tfidf.fit_transform(train_df_featured['note'])

tfidf_df = pd.DataFrame(X_notes_train.toarray(), columns=['note_' + col for col in tfidf.get_feature_names_out()])
final_train_df = pd.concat([train_df_featured.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

# Define final X_full and y_full for training
y_full = final_train_df['readmit_30d']
X_full = final_train_df.drop(columns=['admission_id', 'patient_id', 'readmit_30d', 'note', 'ed_cost_next3y_usd', 'zip3', 'primary_chronic'], errors='ignore')
X_full = X_full.loc[:, ~X_full.columns.duplicated()].fillna(0)
print("✅ Full training dataset ready.")
print("-" * 50)

print("📝 Preparing the official TEST dataset...")
# Prepare Test Data
test_df = pd.merge(admissions_test, patients, on='patient_id', how='left')
test_df = pd.merge(test_df, ed_cost_test, on='patient_id', how='left')
test_df_with_notes = pd.merge(test_df, notes_df, on='admission_id', how='left')
test_df_with_notes['note'] = test_df_with_notes['note'].fillna('')
final_test_df = feature_engineer(test_df_with_notes.copy())
X_notes_test = tfidf.transform(final_test_df['note'])
tfidf_test_df = pd.DataFrame(X_notes_test.toarray(), columns=['note_' + col for col in tfidf.get_feature_names_out()])
final_test_features = pd.concat([final_test_df.reset_index(drop=True), tfidf_test_df.reset_index(drop=True)], axis=1)

# Align columns perfectly and save IDs
test_ids_final = final_test_features['admission_id']
final_test_aligned = final_test_features.reindex(columns=X_full.columns, fill_value=0)
print("✅ Test dataset is ready and aligned.")
print("-" * 50)


# Step 3: Train Final Model, Predict, and Submit
# -------------------------------------------------------------------
print("🤖 Training the final, optimized XGBoost model on ALL data...")
# Use the best parameters we found with Optuna
best_params = {'n_estimators': 957, 'learning_rate': 0.2793, 'max_depth': 8, 'subsample': 0.792, 'colsample_bytree': 0.643, 'gamma': 2.891}
best_params['objective'] = 'binary:logistic'
best_params['eval_metric'] = 'logloss'
best_params['random_state'] = 42
best_params['scale_pos_weight'] = (y_full == 0).sum() / (y_full == 1).sum()

optimized_model = xgb.XGBClassifier(**best_params)
optimized_model.fit(X_full, y_full)
print("✅ Final XGBoost model is trained.")
print("-" * 50)

print("🚀 Generating final predictions...")
final_predictions = optimized_model.predict(final_test_aligned.fillna(0))
submission_df = pd.DataFrame({'admission_id': test_ids_final, 'readmit_30d': final_predictions})
submission_df.to_csv("XGB_ONLY_predictions.csv", index=False)
print("✅ Submission file 'XGB_ONLY_predictions.csv' created.")



✅ Final XGBoost model is trained.
--------------------------------------------------
🚀 Generating final predictions...
✅ Submission file 'XGB_ONLY_predictions.csv' created.


In [4]:
# ============================================================
# Score Estimation via Stratified K-Fold Cross-Validation
# (Test labels are not available on HuggingFace, so we estimate
#  the Macro-F1 score using cross-validation on training data)
# ============================================================
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import f1_score, classification_report

print("📊 Estimating Macro-F1 score via 5-fold Stratified CV on training data...")

best_params_cv = {
    'n_estimators': 957,
    'learning_rate': 0.2793,
    'max_depth': 8,
    'subsample': 0.792,
    'colsample_bytree': 0.643,
    'gamma': 2.891,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42,
    'scale_pos_weight': (y_full == 0).sum() / (y_full == 1).sum()
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_predictions = cross_val_predict(
    xgb.XGBClassifier(**best_params_cv),
    X_full, y_full, cv=cv, n_jobs=-1
)

macro_f1 = f1_score(y_full, cv_predictions, average='macro')
print(f"\n{'='*50}")
print(f"  Estimated Macro-F1 Score: {macro_f1:.4f}")
print(f"{'='*50}")
print(f"\n{classification_report(y_full, cv_predictions, target_names=['No Readmit', 'Readmit'])}")


  Estimated Macro-F1 Score: 0.8456

              precision    recall  f1-score   support

  No Readmit       0.85      0.84      0.84      2479
     Readmit       0.84      0.85      0.85      2521

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [5]:
"""
Improved Stacking - Simple & Robust Approach to Beat 0.8960

Focus on what works:
1. More diverse XGBoost configurations
2. Enhanced features (proven ones)
3. Better meta-learner with probability stacking
4. No complex feature selection (simpler = more robust)
"""

import pandas as pd
import numpy as np
import xgboost as xgb
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from huggingface_hub import hf_hub_download
import warnings

warnings.filterwarnings('ignore')

def improved_feature_engineering(df):
    """
    Feature engineering with proven high-value features.
    """
    # One-hot encoding
    categorical_cols = ['sex', 'insurance', 'primary_dx']
    df = pd.get_dummies(df, columns=categorical_cols, dummy_na=False, dtype=int)
    
    # Core temporal features
    df['is_weekend_discharge'] = df['discharge_weekday'].isin([6, 7]).astype(int)
    df['is_monday'] = (df['discharge_weekday'] == 1).astype(int)
    df['is_friday'] = (df['discharge_weekday'] == 5).astype(int)
    
    # Cost features
    df['avg_cost_per_ed_visit'] = df['prior_ed_cost_5y_usd'] / (df['prior_ed_visits_5y'] + 1)
    df['avg_cost_per_ed_visit'] = df['avg_cost_per_ed_visit'].replace([np.inf, -np.inf], 0).fillna(0)
    
    # Age features
    df['is_elderly'] = (df['age'] >= 65).astype(int)
    df['is_very_elderly'] = (df['age'] >= 80).astype(int)
    df['age_squared'] = df['age'] ** 2
    
    # Charlson features
    df['high_charlson'] = (df['charlson_band'] >= 3).astype(int)
    df['charlson_squared'] = df['charlson_band'] ** 2
    
    # ED visit features
    df['recent_ed_ratio'] = df['ed_visits_6m'] / (df['prior_ed_visits_5y'] + 1)
    df['is_frequent_ed'] = (df['ed_visits_6m'] >= 2).astype(int)
    df['is_very_frequent_ed'] = (df['ed_visits_6m'] >= 3).astype(int)
    df['is_new_patient'] = (df['prior_ed_visits_5y'] == 0).astype(int)
    
    # LOS features
    df['los_log'] = np.log1p(df['los_days'])
    df['is_long_stay'] = (df['los_days'] >= 5).astype(int)
    
    # Key interactions
    df['age_x_charlson'] = df['age'] * df['charlson_band']
    df['age_x_ed'] = df['age'] * df['ed_visits_6m']
    df['charlson_x_ed'] = df['charlson_band'] * df['ed_visits_6m']
    df['los_x_ed'] = df['los_days'] * df['ed_visits_6m']
    
    # Risk scores
    df['risk_score_1'] = (
        df['age'] / 100 + 
        df['charlson_band'] * 0.5 + 
        df['ed_visits_6m'] * 0.3
    )
    
    df['risk_score_2'] = (
        df['is_elderly'] * 2 +
        df['high_charlson'] * 3 +
        df['is_frequent_ed'] * 2
    )
    
    return df

def main():
    """
    Improved stacking to beat 0.8960.
    """
    print("=" * 70)
    print("IMPROVED STACKING - Target: >0.90 F1")
    print("=" * 70)
    
    # Check GPU
    build_info = xgb.build_info()
    gpu_available = build_info.get('USE_CUDA', False)
    print(f"\n{'✅ GPU Enabled' if gpu_available else '⚠️  CPU Mode'}")
    
    # Load data
    print("\n📂 Loading data...")
    REPO_ID = "lainmn/AgentDS-Healthcare"
    
    admissions_train = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/admissions_train.csv", repo_type="dataset"))
    patients = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/patients.csv", repo_type="dataset"))
    ed_cost_train = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/ed_cost_train.csv", repo_type="dataset"))
    admissions_test = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/admissions_test.csv", repo_type="dataset"))
    ed_cost_test = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/ed_cost_test.csv", repo_type="dataset"))
    
    with open(hf_hub_download(REPO_ID, "Healthcare/discharge_notes.json", repo_type="dataset"), 'r') as f:
        discharge_notes = json.load(f)
    notes_df = pd.DataFrame(discharge_notes)
    
    # Prepare training data
    print("\n🔧 Feature engineering...")
    train_df = pd.merge(admissions_train, patients, on='patient_id', how='left')
    train_df = pd.merge(train_df, ed_cost_train, on='patient_id', how='left')
    train_df = pd.merge(train_df, notes_df, on='admission_id', how='left')
    train_df['note'] = train_df['note'].fillna('')
    
    train_df_featured = improved_feature_engineering(train_df.copy())
    
    # TF-IDF - increase to 850 features with trigrams
    print("   Creating enhanced TF-IDF (850 features with trigrams)...")
    tfidf = TfidfVectorizer(
        max_features=850,
        stop_words='english',
        ngram_range=(1, 3),  # Trigrams for better context
        min_df=3,
        max_df=0.95,
        sublinear_tf=True
    )
    
    X_notes_train = tfidf.fit_transform(train_df_featured['note'])
    tfidf_df = pd.DataFrame(
        X_notes_train.toarray(),
        columns=['note_' + col for col in tfidf.get_feature_names_out()]
    )
    
    final_train_df = pd.concat([
        train_df_featured.reset_index(drop=True),
        tfidf_df.reset_index(drop=True)
    ], axis=1)
    
    y_full = final_train_df['readmit_30d']
    X_full = final_train_df.drop(columns=[
        'admission_id', 'patient_id', 'readmit_30d', 'note',
        'ed_cost_next3y_usd', 'zip3', 'primary_chronic'
    ], errors='ignore')
    
    X_full = X_full.loc[:, ~X_full.columns.duplicated()].fillna(0)
    print(f"✅ Features ready: {X_full.shape}")
    
    # Prepare test data
    print("\n📝 Preparing test data...")
    test_df = pd.merge(admissions_test, patients, on='patient_id', how='left')
    test_df = pd.merge(test_df, ed_cost_test, on='patient_id', how='left')
    test_df = pd.merge(test_df, notes_df, on='admission_id', how='left')
    test_df['note'] = test_df['note'].fillna('')
    
    final_test_df = improved_feature_engineering(test_df.copy())
    
    X_text_test = tfidf.transform(final_test_df['note'])
    tfidf_test_df = pd.DataFrame(
        X_text_test.toarray(),
        columns=['note_' + col for col in tfidf.get_feature_names_out()]
    )
    
    final_test_features = pd.concat([
        final_test_df.reset_index(drop=True),
        tfidf_test_df.reset_index(drop=True)
    ], axis=1)
    
    test_ids = final_test_features['admission_id']
    final_test_aligned = final_test_features.reindex(columns=X_full.columns, fill_value=0).fillna(0)
    print(f"✅ Test data ready: {final_test_aligned.shape}")
    
    # Build improved stacking ensemble
    print("\n🤖 Building Improved Stacking Ensemble...")
    print("   5 diverse base models with 8-fold CV")
    
    scale_pos_weight = (y_full == 0).sum() / (y_full == 1).sum()
    
    # Base Model 1: Your optimized XGBoost
    xgb_opt = Pipeline([
        ('model', xgb.XGBClassifier(
            n_estimators=771,
            learning_rate=0.03002156989594884,
            max_depth=7,
            subsample=0.6844646638528962,
            colsample_bytree=0.8063408933100336,
            gamma=3.418014628166754,
            objective='binary:logistic',
            random_state=42,
            device='cuda' if gpu_available else 'cpu',
            tree_method='hist',
            scale_pos_weight=scale_pos_weight
        ))
    ])
    
    # Base Model 2: XGBoost - more trees, lower learning rate
    xgb_deep = Pipeline([
        ('model', xgb.XGBClassifier(
            n_estimators=1500,
            learning_rate=0.015,
            max_depth=6,
            subsample=0.75,
            colsample_bytree=0.85,
            gamma=2.0,
            min_child_weight=2,
            objective='binary:logistic',
            random_state=999,
            device='cuda' if gpu_available else 'cpu',
            tree_method='hist',
            scale_pos_weight=scale_pos_weight
        ))
    ])
    
    # Base Model 3: XGBoost - shallow and wide
    xgb_shallow = Pipeline([
        ('model', xgb.XGBClassifier(
            n_estimators=1200,
            learning_rate=0.02,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.9,
            gamma=1.0,
            objective='binary:logistic',
            random_state=555,
            device='cuda' if gpu_available else 'cpu',
            tree_method='hist',
            scale_pos_weight=scale_pos_weight
        ))
    ])
    
    # Base Model 4: Logistic Regression with L2
    lr_l2 = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(
            C=0.5,
            class_weight='balanced',
            solver='saga',
            penalty='l2',
            max_iter=1000,
            random_state=42
        ))
    ])
    
    # Base Model 5: Logistic Regression with L1 (different features selected)
    lr_l1 = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(
            C=0.3,
            class_weight='balanced',
            solver='saga',
            penalty='l1',
            max_iter=1000,
            random_state=777
        ))
    ])
    
    # Define base models
    estimators = [
        ('xgb_opt', xgb_opt),
        ('xgb_deep', xgb_deep),
        ('xgb_shallow', xgb_shallow),
        ('lr_l2', lr_l2),
        ('lr_l1', lr_l1)
    ]
    
    # Meta-learner: Logistic Regression
    meta_learner = LogisticRegression(
        C=1.5,
        class_weight='balanced',
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    )
    
    # Create stacking classifier with 8-fold CV
    stacker = StackingClassifier(
        estimators=estimators,
        final_estimator=meta_learner,
        cv=StratifiedKFold(n_splits=8, shuffle=True, random_state=42),
        stack_method='predict_proba',
        passthrough=False,
        n_jobs=1,
        verbose=2
    )
    
    print("✅ Ensemble configured: 5 models × 8 folds = 40 base models + meta-learner")
    print("-" * 70)
    
    # Train
    print("\n🚀 Training Improved Stacking Ensemble...")
    print("   Expected time: 15-25 minutes")
    print()
    
    stacker.fit(X_full, y_full)
    
    print("\n✅ Training complete!")
    
    # Generate predictions
    print("\n📤 Generating predictions...")
    final_predictions = stacker.predict(final_test_aligned)
    
    # Get probabilities
    final_probas = stacker.predict_proba(final_test_aligned)[:, 1]
    
    submission_df = pd.DataFrame({
        'admission_id': test_ids,
        'readmit_30d': final_predictions
    })
    
    submission_filename = 'readmission_predictions_improved.csv'
    submission_df.to_csv(submission_filename, index=False)
    
    # Save probabilities
    proba_df = pd.DataFrame({
        'admission_id': test_ids,
        'probability': final_probas
    })
    proba_df.to_csv('readmission_probabilities.csv', index=False)
    
    print("\n" + "=" * 70)
    print("✅ IMPROVED STACKING COMPLETE!")
    print("=" * 70)
    print(f"Submission file: '{submission_filename}'")
    print(f"Probabilities: 'readmission_probabilities.csv'")
    print(f"Total predictions: {len(final_predictions)}")
    print(f"Predicted readmissions: {final_predictions.sum()} ({100*final_predictions.mean():.2f}%)")
    print(f"\nYour baseline: 0.8960")
    print(f"Expected: 0.900-0.910 F1")
    print("\n💡 Key Improvements:")
    print("   • 850 TF-IDF features (vs 750) with trigrams")
    print("   • 5 diverse base models (3 XGBoost + 2 LR)")
    print("   • 8-fold CV for stable meta-model")
    print("   • Enhanced features (age², charlson², risk scores)")
    print("   • L1 + L2 regularized LR for feature diversity")
    print("=" * 70)

if __name__ == "__main__":
    main()

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.8min finished


In [6]:
# ============================================================
# Score Estimation for Improved Stacking Ensemble
# via Stratified K-Fold Cross-Validation
# ============================================================
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline

print("📊 Estimating Macro-F1 for Improved Stacking Ensemble via 5-fold CV...")
print("   (This will take several minutes as it trains the full ensemble per fold)\n")

# Re-use X_full, y_full from the stacking cell's main() scope
# Re-run data prep inline to ensure variables are available
REPO_ID = "lainmn/AgentDS-Healthcare"
admissions_train = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/admissions_train.csv", repo_type="dataset"))
patients = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/patients.csv", repo_type="dataset"))
ed_cost_train = pd.read_csv(hf_hub_download(REPO_ID, "Healthcare/ed_cost_train.csv", repo_type="dataset"))

with open(hf_hub_download(REPO_ID, "Healthcare/discharge_notes.json", repo_type="dataset"), 'r') as f:
    discharge_notes_cv = json.load(f)
notes_df_cv = pd.DataFrame(discharge_notes_cv)

train_df_cv = pd.merge(admissions_train, patients, on='patient_id', how='left')
train_df_cv = pd.merge(train_df_cv, ed_cost_train, on='patient_id', how='left')
train_df_cv = pd.merge(train_df_cv, notes_df_cv, on='admission_id', how='left')
train_df_cv['note'] = train_df_cv['note'].fillna('')

train_df_cv_feat = improved_feature_engineering(train_df_cv.copy())

tfidf_cv = TfidfVectorizer(
    max_features=850, stop_words='english', ngram_range=(1, 3),
    min_df=3, max_df=0.95, sublinear_tf=True
)
X_notes_cv = tfidf_cv.fit_transform(train_df_cv_feat['note'])
tfidf_cv_df = pd.DataFrame(X_notes_cv.toarray(), columns=['note_' + c for c in tfidf_cv.get_feature_names_out()])
final_cv_df = pd.concat([train_df_cv_feat.reset_index(drop=True), tfidf_cv_df.reset_index(drop=True)], axis=1)

y_cv = final_cv_df['readmit_30d']
X_cv = final_cv_df.drop(columns=['admission_id', 'patient_id', 'readmit_30d', 'note',
                                   'ed_cost_next3y_usd', 'zip3', 'primary_chronic'], errors='ignore')
X_cv = X_cv.loc[:, ~X_cv.columns.duplicated()].fillna(0)

scale_pos_weight_cv = (y_cv == 0).sum() / (y_cv == 1).sum()
build_info = xgb.build_info()
gpu_available = build_info.get('USE_CUDA', False)
device = 'cuda' if gpu_available else 'cpu'

# Build the same stacking ensemble
estimators_cv = [
    ('xgb_opt', Pipeline([('model', xgb.XGBClassifier(
        n_estimators=771, learning_rate=0.03, max_depth=7, subsample=0.684,
        colsample_bytree=0.806, gamma=3.418, objective='binary:logistic',
        random_state=42, device=device, tree_method='hist', scale_pos_weight=scale_pos_weight_cv))])),
    ('xgb_deep', Pipeline([('model', xgb.XGBClassifier(
        n_estimators=1500, learning_rate=0.015, max_depth=6, subsample=0.75,
        colsample_bytree=0.85, gamma=2.0, min_child_weight=2, objective='binary:logistic',
        random_state=999, device=device, tree_method='hist', scale_pos_weight=scale_pos_weight_cv))])),
    ('xgb_shallow', Pipeline([('model', xgb.XGBClassifier(
        n_estimators=1200, learning_rate=0.02, max_depth=5, subsample=0.8,
        colsample_bytree=0.9, gamma=1.0, objective='binary:logistic',
        random_state=555, device=device, tree_method='hist', scale_pos_weight=scale_pos_weight_cv))])),
    ('lr_l2', Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(
        C=0.5, class_weight='balanced', solver='saga', penalty='l2', max_iter=1000, random_state=42))])),
    ('lr_l1', Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(
        C=0.3, class_weight='balanced', solver='saga', penalty='l1', max_iter=1000, random_state=777))])),
]

stacker_cv = StackingClassifier(
    estimators=estimators_cv,
    final_estimator=LogisticRegression(C=1.5, class_weight='balanced', solver='lbfgs', max_iter=1000, random_state=42),
    cv=StratifiedKFold(n_splits=8, shuffle=True, random_state=42),
    stack_method='predict_proba', passthrough=False, n_jobs=1
)

# Outer 5-fold CV to estimate generalization score
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_preds_stacking = cross_val_predict(stacker_cv, X_cv, y_cv, cv=outer_cv, n_jobs=1)

macro_f1_stacking = f1_score(y_cv, cv_preds_stacking, average='macro')
print(f"\n{'='*60}")
print(f"  Simple XGBoost Macro-F1 (from earlier):  0.8456")
print(f"  Stacking Ensemble Macro-F1 (5-fold CV):  {macro_f1_stacking:.4f}")
print(f"{'='*60}")
print(f"\n{classification_report(y_cv, cv_preds_stacking, target_names=['No Readmit', 'Readmit'])}")


  Simple XGBoost Macro-F1 (from earlier):  0.8456
  Stacking Ensemble Macro-F1 (5-fold CV):  0.8956

              precision    recall  f1-score   support

  No Readmit       0.90      0.89      0.89      2479
     Readmit       0.90      0.90      0.90      2521

    accuracy                           0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000

